<a href="https://colab.research.google.com/github/Gabrieldiasdeoliveira/cross_up_selling/blob/main/cross_up_selling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analise de dados loja online

*Nesse notebock vou desenvolver uma analise em cima de um dataset que contém informações sobre transações de compras em um supermercado online, incluindo quais produtos foram comprados juntos.
A idéia é analisar cross-selling e upselling com base nos padrões de compras dos clientes.*


#Dowload das bases do kaggle

Nessa etava vamos acessar o dataset e fazer o dowload das bases

In [1]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle (8).json


{'kaggle (8).json': b'{"username":"gabrielestatistico","key":"0c5e3e674729d6c445c0e7b78b9c74ea"}'}

In [3]:
# pip install kaggle -q

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
#!mv kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c instacart-market-basket-analysis


instacart-market-basket-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
# extraindo todos os arquivos do dataset
!unzip -o instacart-market-basket-analysis.zip


Archive:  instacart-market-basket-analysis.zip
  inflating: aisles.csv.zip          
  inflating: departments.csv.zip     
  inflating: order_products__prior.csv.zip  
  inflating: order_products__train.csv.zip  
  inflating: orders.csv.zip          
  inflating: products.csv.zip        
  inflating: sample_submission.csv.zip  


In [10]:
# Descompactando os arquivos
import zipfile
import os

# Lista de arquivos .csv.zip para descompactar
zip_files = [
    "aisles.csv.zip",
    "departments.csv.zip",
    "order_products__prior.csv.zip",
    "order_products__train.csv.zip",
    "orders.csv.zip",
    "products.csv.zip",
    "sample_submission.csv.zip"
]

# Descompacta cada arquivo .csv.zip
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall()


In [ ]:
import pandas as pd

# Leitura dos arquivos CSV
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
sample_submission = pd.read_csv('sample_submission.csv')




# Visualização

In [13]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("aisles:")
print(aisles.head(), "\n")


aisles:
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation 



In [14]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("departments:")
print(departments.head(), "\n")

departments:
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol 



In [15]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("order_products_prior:")
print(order_products_prior.head(), "\n")

order_products_prior:
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0 



In [37]:
# Contar a quantidade de order_id distintos na base 'merged_with_orders'
distinct_order_ids = order_products_prior['order_id'].nunique()
distinct_order_ids


3214874

In [16]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("order_products_train:")
print(order_products_train.head(), "\n")

order_products_train:
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1 



In [17]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("orders:")
print(orders.head(), "\n")

orders:
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0   



In [18]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("products:")
print(products.head(), "\n")

products:
   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13   



In [19]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("sample_submission:")
print(sample_submission.head(), "\n")

sample_submission:
   order_id     products
0        17  39276 29259
1        34  39276 29259
2       137  39276 29259
3       182  39276 29259
4       257  39276 29259 



# Analise Exploratória de dados

*Nessa etapa faremos algumas analises descritivas, categorizações e clusterizações para entender alguns padrões nos dados.*

*Inicialmente vamos entender a recorrencia nas vendas dos produtos*


In [12]:
# Passo 1: Realizar a junção
merged_data = pd.merge(products, aisles, on='aisle_id', how='left')

# Passo 2: Contar produtos por aisle
aisle_counts = merged_data['aisle'].value_counts().reset_index()
aisle_counts.columns = ['aisle', 'product_count']  # Renomeia as colunas para melhor legibilidade

# Exibir contagem por aisle
print("Contagem de produtos por aisle:")
print(aisle_counts)

Contagem de produtos por aisle:
                            aisle  product_count
0                         missing           1258
1                 candy chocolate           1246
2                   ice cream ice           1091
3            vitamins supplements           1038
4                          yogurt           1026
..                            ...            ...
129                  frozen juice             47
130              baby accessories             44
131              packaged produce             32
132  bulk grains rice dried goods             26
133  bulk dried fruits vegetables             12

[134 rows x 2 columns]


In [13]:
merged_with_departments = pd.merge(merged_data, departments, on='department_id', how='left')

# Agora, contar produtos por department
department_counts = merged_with_departments['department'].value_counts().reset_index()
department_counts.columns = ['Departamento', 'Quantidade de Produtos']

# Exibir contagem por department
print("\nContagem de produtos por department:")
print(department_counts)


Contagem de produtos por department:
       Departamento  Quantidade de Produtos
0     personal care                    6563
1            snacks                    6264
2            pantry                    5371
3         beverages                    4365
4            frozen                    4007
5        dairy eggs                    3449
6         household                    3085
7      canned goods                    2092
8   dry goods pasta                    1858
9           produce                    1684
10           bakery                    1516
11             deli                    1322
12          missing                    1258
13    international                    1139
14        breakfast                    1115
15           babies                    1081
16          alcohol                    1054
17             pets                     972
18     meat seafood                     907
19            other                     548
20             bulk                   

In [15]:
import pandas as pd

# Passo 1: Carregar os dados
products = pd.read_csv('products.csv')          # Carregue o arquivo com dados de produtos
aisles = pd.read_csv('aisles.csv')              # Carregue o arquivo com dados de corredores (aisles)
departments = pd.read_csv('departments.csv')    # Carregue o arquivo com dados de departamentos
orders_prior = pd.read_csv('order_products__prior.csv')  # Carregue os pedidos anteriores
orders_train = pd.read_csv('order_products__train.csv')  # Carregue os pedidos de treino




In [16]:
# Passo 3: Realizar a junção
merged_data = pd.merge(products, aisles, on='aisle_id', how='left')
merged_with_departments = pd.merge(merged_data, departments, on='department_id', how='left')
merged_with_orders = pd.merge(merged_with_departments, order_products_prior, on='product_id', how='left')

# Passo 4: Agrupar e contar as vendas por departamento e produto
product_sales = merged_with_orders.groupby(['department', 'product_name'])['order_id'].count().reset_index()

# Passo 5: Renomear a coluna para melhor legibilidade
product_sales.columns = ['department', 'product_name', 'total_sales']

# Passo 6: Ordenar os produtos por departamento e total de vendas
product_sales = product_sales.sort_values(by=['department', 'total_sales'], ascending=[True, False])

# Passo 7: Selecionar os produtos mais vendidos em cada departamento
#top_products_by_department = product_sales.groupby('department').head(10)

In [17]:
# Passo 2: Concatenar os dados de vendas
orders_combined = pd.concat([orders_prior, orders_train], ignore_index=True)

In [19]:
# Passo 3: Realizar as junções necessárias
# merged_data = pd.merge(products, aisles, on='aisle_id', how='left')
# merged_with_departments = pd.merge(merged_data, departments, on='department_id', how='left')
# merged_with_orders = pd.merge(merged_with_departments, orders_combined, on='product_id', how='left')

In [20]:
# Incluir user_id no merged_with_orders
if 'user_id' not in merged_with_orders.columns:
    orders = pd.read_csv('orders.csv')
    merged_with_orders = pd.merge(merged_with_orders, orders[['order_id', 'user_id']], on='order_id', how='left')

In [21]:
# Inspecionar as colunas do DataFrame merged_with_orders
print("Colunas do DataFrame merged_with_orders:")
print(merged_with_orders.columns)

Colunas do DataFrame merged_with_orders:
Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'aisle',
       'department', 'order_id', 'add_to_cart_order', 'reordered', 'user_id'],
      dtype='object')


In [22]:
# Passo 4: Agrupar e contar as vendas por departamento e produto
product_sales = merged_with_orders.groupby(['department', 'product_id', 'product_name'])['order_id'].count().reset_index()
product_sales.columns = ['department', 'product_id', 'product_name', 'total_sales']

In [23]:
# Ordenar a tabela 'product_sales' pela coluna 'total_sales' do maior para o menor
product_sales = product_sales.sort_values(by='total_sales', ascending=False)

# Exibir as primeiras linhas da tabela ordenada
product_sales

,department,product_id,product_name,total_sales
42610,produce,24852,Banana,472565
42226,produce,13176,Bag of Organic Bananas,379450
42495,produce,21137,Organic Strawberries,264683
42524,produce,21903,Organic Baby Spinach,241921
43337,produce,47209,Organic Hass Avocado,213584
...,...,...,...,...
27921,missing,36233,Water With Electrolytes,0
46622,snacks,25383,Chocolate Go Bites,0
7759,beverages,46625,Single Barrel Kentucky Straight Bourbon Whiskey,0
11511,dairy eggs,3718,Wasabi Cheddar Spreadable Cheese,0


In [24]:
# Passo 7: Contar compradores únicos para cada produto
unique_buyers = merged_with_orders.groupby('product_id')['order_id'].nunique().reset_index()
unique_buyers.columns = ['product_id', 'unique_buyers']

In [25]:
# Passo 8: Contar produtos reordenados e comprados pela primeira vez
reorder_count = merged_with_orders.groupby('product_id')['reordered'].sum().reset_index()
reorder_count.columns = ['product_id', 'reorder_count']


In [26]:
product_sales

,department,product_id,product_name,total_sales
42610,produce,24852,Banana,472565
42226,produce,13176,Bag of Organic Bananas,379450
42495,produce,21137,Organic Strawberries,264683
42524,produce,21903,Organic Baby Spinach,241921
43337,produce,47209,Organic Hass Avocado,213584
...,...,...,...,...
27921,missing,36233,Water With Electrolytes,0
46622,snacks,25383,Chocolate Go Bites,0
7759,beverages,46625,Single Barrel Kentucky Straight Bourbon Whiskey,0
11511,dairy eggs,3718,Wasabi Cheddar Spreadable Cheese,0


In [27]:
# Passo 9: Calcular a porcentagem de reordenação
# Merge product_sales with reorder_count on 'product_id' to add 'reorder_count' column
product_sales = product_sales.merge(reorder_count, on='product_id', how='left')

# Now you can calculate the reorder percentage
product_sales['reorder_percentage'] = (product_sales['reorder_count'] / product_sales['total_sales']) * 100

In [28]:
# Passo 10: Adicionar os dados de compradores únicos e recompra ao DataFrame dos produtos mais vendidos
product_sales = product_sales.merge(unique_buyers, on='product_id', how='left')
product_sales = product_sales.merge(reorder_count, on='product_id', how='left')

In [29]:
product_sales

,department,product_id,product_name,total_sales,reorder_count_x,reorder_percentage,unique_buyers,reorder_count_y
0,produce,24852,Banana,472565,398609.0,84.350089,472565,398609.0
1,produce,13176,Bag of Organic Bananas,379450,315913.0,83.255501,379450,315913.0
2,produce,21137,Organic Strawberries,264683,205845.0,77.770389,264683,205845.0
3,produce,21903,Organic Baby Spinach,241921,186884.0,77.250011,241921,186884.0
4,produce,47209,Organic Hass Avocado,213584,170131.0,79.655311,213584,170131.0
...,...,...,...,...,...,...,...,...
49683,missing,36233,Water With Electrolytes,0,0.0,NaN,0,0.0
49684,snacks,25383,Chocolate Go Bites,0,0.0,NaN,0,0.0
49685,beverages,46625,Single Barrel Kentucky Straight Bourbon Whiskey,0,0.0,NaN,0,0.0
49686,dairy eggs,3718,Wasabi Cheddar Spreadable Cheese,0,0.0,NaN,0,0.0


In [30]:
# Passo 1: Selecionar as colunas desejadas
selected_columns = ['department', 'product_id', 'product_name', 'total_sales', 'reorder_count_x', 'reorder_percentage']

# Passo 2: Manter apenas as colunas selecionadas
product_sales = product_sales[selected_columns]

# Passo 3: Ordenar pela coluna 'reorder_percentage' de forma descendente
product_sales = product_sales.sort_values(by='reorder_percentage', ascending=False)

In [31]:
product_sales

,department,product_id,product_name,total_sales,reorder_count_x,reorder_percentage
23587,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647
21201,personal care,2075,Serenity Ultimate Extrema Overnight Pads,87,81.0,93.103448
40089,beverages,43553,Orange Energy Shots,13,12.0,92.307692
19946,snacks,27740,Chocolate Love Bar,101,93.0,92.079208
30044,babies,14609,Soy Powder Infant Formula,35,32.0,91.428571
...,...,...,...,...,...,...
49683,missing,36233,Water With Electrolytes,0,0.0,NaN
49684,snacks,25383,Chocolate Go Bites,0,0.0,NaN
49685,beverages,46625,Single Barrel Kentucky Straight Bourbon Whiskey,0,0.0,NaN
49686,dairy eggs,3718,Wasabi Cheddar Spreadable Cheese,0,0.0,NaN


In [32]:
# Converter a coluna 'total_sales' para numérico, forçando erros para NaN
product_sales['total_sales'] = pd.to_numeric(product_sales['total_sales'], errors='coerce')

# Remover as linhas onde 'total_sales' é 0 ou nulo
product_sales = product_sales[product_sales['total_sales'] != 0.0]


In [33]:
# Selecionando as colunas desejadas
selected_columns = ['department', 'product_id', 'product_name', 'total_sales', 'reorder_count', 'reorder_percentage']
# Renomear a coluna reorder_count_x para reorder_count
product_sales = product_sales.rename(columns={'reorder_count_x': 'reorder_count'})
# Exibir apenas as colunas selecionadas
product_sales = product_sales[selected_columns]

In [34]:
product_sales

,department,product_id,product_name,total_sales,reorder_count,reorder_percentage
23587,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647
21201,personal care,2075,Serenity Ultimate Extrema Overnight Pads,87,81.0,93.103448
40089,beverages,43553,Orange Energy Shots,13,12.0,92.307692
19946,snacks,27740,Chocolate Love Bar,101,93.0,92.079208
30044,babies,14609,Soy Powder Infant Formula,35,32.0,91.428571
...,...,...,...,...,...,...
46447,pantry,15909,"Any Day Chicken Blend, All Natural Spice Blend",6,0.0,0.000000
46448,deli,14391,Herb Salami,6,0.0,0.000000
40691,personal care,30626,Calcium Magnesium Zinc,12,0.0,0.000000
40681,meat seafood,38989,Naturally Smoked Atlantic Salmon,12,0.0,0.000000


*Incluindo a o numero do pedido para conseguirmos agrupar depois.*

In [35]:
# Passo 1: Fazer o merge entre 'top_products_by_department' e 'order_products_prior' para incluir o 'order_id' de cada produto
merged_with_orders = pd.merge(product_sales, order_products_prior[['order_id', 'product_id']], on='product_id', how='left')

# Passo 2: Verificar o resultado para garantir que cada produto tem o 'order_id' correto
merged_with_orders


,department,product_id,product_name,total_sales,reorder_count,reorder_percentage,order_id
0,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647,31818
1,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647,143173
2,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647,143998
3,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647,154038
4,deli,6433,Raw Veggie Wrappers,68,64.0,94.117647,154129
...,...,...,...,...,...,...,...
32434484,meat seafood,38989,Naturally Smoked Atlantic Salmon,12,0.0,0.000000,2342383
32434485,meat seafood,38989,Naturally Smoked Atlantic Salmon,12,0.0,0.000000,2697668
32434486,meat seafood,38989,Naturally Smoked Atlantic Salmon,12,0.0,0.000000,2899637
32434487,meat seafood,38989,Naturally Smoked Atlantic Salmon,12,0.0,0.000000,2976626


In [36]:
# Exibindo todas as linhas do DataFrame filtrado
filtered_orders1 = order_products_prior[order_products_prior['order_id'] == 1728]
filtered_orders1


,order_id,product_id,add_to_cart_order,reordered
16484,1728,32455,1,0
16485,1728,18023,2,0
16486,1728,39275,3,0
16487,1728,20776,4,0


In [37]:
# Ajustar a configuração do pandas para exibir todas as linhas
pd.set_option('display.max_rows', None)

# Exibindo todas as linhas do DataFrame filtrado
filtered_orders = merged_with_orders[merged_with_orders['order_id'] == 1728]
filtered_orders



,department,product_id,product_name,total_sales,reorder_count,reorder_percentage,order_id
4451311,beverages,18023,Organic Coconut Water,5474,4083.0,74.588966,1728
11722586,snacks,32455,Whole Grain Cheddar Baked Snack Crackers,7321,4807.0,65.660429,1728
14362118,produce,39275,Organic Blueberries,100060,62922.0,62.884269,1728
30928465,dry goods pasta,20776,Organic Quinoa,1597,463.0,28.991860,1728


# Analise Descritiva e Inferencial



### 1. Analisar quais produtos são comprados juntos
Tipo de análise: Análise de associação ou análise de padrões (Data Mining).

*Aqui, nós vamos basicamente identificar padrões de compra entre diferentes produtos.*

---

### 2. Visualizar as tendências de compra
*Tipo de análise: Análise descritiva.               
Nessa análise, basicamente estaremos visualizando os padrões de compra com gráficos, como distribuição de produtos comprados por pedido, e pode também usar métodos de agregação para entender melhor os padrões de compra.*

---


In [38]:
# Contar a quantidade de order_id distintos na base 'merged_with_orders'
distinct_order_ids = merged_with_orders['order_id'].nunique()
distinct_order_ids


3214874

In [128]:
# Encontrando os order_id que estão em 'order_products_prior' mas não estão em 'merged_with_orders'

# Obter os order_id únicos de ambas as bases
order_products_prior_ids = order_products_prior['order_id'].unique()
merged_with_orders_ids = merged_with_orders['order_id'].unique()

# Identificar os order_id presentes em 'order_products_prior' mas não em 'merged_with_orders'
missing_order_ids = set(order_products_prior_ids) - set(merged_with_orders_ids)

# Exibir os order_id faltantes
missing_order_ids


set()

In [39]:
print(merged_with_orders)

KeyboardInterrupt: 

### 3. Aprofundar-se nas métricas de recorrência

*Tipo de análise: Análise inferencial.*

---